# An Example Jupyter Notebook for Greylevel Analysis of Active Cathode Particles in X-ray CT tomograms 
Here we present a method for analysing the greylevel change in active cathode particles in their pristine state and after ageing. When aged Nickel Manganese Cobalt particles crack, resulting in changes in greylevel when imaged with X-ray CT. Typically, particles crack more in the center than at the surface. In this notebook we demonstrate how to use the GRAPES (Grey-level Analysis of Particles) toolkit (`utils/GRAPES.py`) to analyse this phenomenon.

In [ ]:
import os, sys
CWD = os.getcwd()
if os.path.basename(CWD) == "notebooks":
    PROJECT_ROOT = os.path.abspath(os.path.join(CWD, ".."))
else:
    PROJECT_ROOT = CWD  # fallback if already at root
os.chdir(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "utils"))
print(f"Correct Working Directory: {str(os.path.basename(os.getcwd()))=='battery_xct_notebooks'}")

import tifffile as tiff
import numpy as np
import numpy.ma as ma
%matplotlib widget
from utils.plotting_utils import view_axis0, view_axis0_with_labels
import utils.GRAPES as gp
from IPython.display import display

First we explore the dataset. We can scroll through the original data and particle masks using the widgets below.

In [ ]:
pristine_intensity = tiff.imread('data/prist_gl.tif')
pristine_labels = tiff.imread('data/prist_lbl.tif')
pristine_labels = np.repeat(pristine_labels, 2, axis=0)[:-1] # image was downsampled in axis zero for repo, resizes to original aspect

slider = view_axis0_with_labels(pristine_intensity, pristine_labels)
display(slider)

In [ ]:
aged_intensity = tiff.imread('data/charge_gl.tif')
aged_labels = tiff.imread('data/charge_lbl.tif')
aged_labels = np.repeat(aged_labels, 2, axis=0)[:-1] # image was downsampled in axis zero for repo, resizes to original aspect

slider1 = view_axis0_with_labels(aged_intensity, aged_labels)
display(slider1)

The particle segmentations were achieved using `skimage.segmentation.watershed` with a smoothed distance map. The segmentation workflow is excluded, but is available from the authors on request.

Using the `GRAPES` toolkit we can calculate a dataframe of particles and particle properties. The GRAPES toolkit has been previously discussed in a paper [here](https://onlinelibrary.wiley.com/doi/full/10.1002/smtd.202500082).

In [ ]:
pristine_df = gp.GRAPES(pristine_labels, pristine_intensity)
aged_df = gp.GRAPES(aged_labels, aged_intensity)

In [ ]:
pristine_df.head()

We can now reduce the data into volume quartiles 

In [ ]:
pristine_qdf = gp.radial_layers_quartiles(pristine_df, prop = 'volume')
charged_qdf = gp.radial_layers_quartiles(aged_df, prop = 'volume')

We can now plot the mean normalised greylevel value in radial layers averaged across all the particles in the dataset. We see that the particles are brighter at the surface, this is due to a phase contrast effect. 

In [ ]:
gp.plot_quartile_radial_layers(pristine_qdf, prop = 'radial_layers_graylevelnormed')

In [ ]:
gp.plot_quartile_radial_layers(charged_qdf, prop = 'radial_layers_graylevelnormed')

By comparing the plots for `pristine_intensity' vs 'aged_intensity', we can see that the aged particles had lower intensities on average in the centers of the particles. This is characteristic of the cracking that occurs during ageing. 

Next we demonstrate how the `GRAPES` toolkit can be used to visualise the distribution of a property in space. Using `props_2_image` we can replot a property from the dataframe in image space. This is demosntrated below for 'intensity_mean' of each particle.

In the plot below we suggest choosing the `viridis` colormap and moving the 'range' slider up so that the intensity difference is viewable.

In [ ]:
pristine_mean_int = gp.prop_2_image(pristine_labels, pristine_df, prop = 'intensity_mean')
slider2 = view_axis0(ma.masked_where(pristine_mean_int == 0, pristine_mean_int).data)
display(slider2)

In [ ]:
charged_mean_int = gp.prop_2_image(aged_labels, aged_df, prop = 'intensity_mean')
slider3 = view_axis0(charged_mean_int)
display(slider3)

### Conclusion
The `GRAPES` toolkit can be used characterize greylevel change and other particle level characteristics in X-ray CT datasets of particles. In this case relatively small 256 x 256 x 256 volumes were used, however, this method works best with larger volumes. For a more detailed description of the electrochemical imaging use-case see this [paper](https://onlinelibrary.wiley.com/doi/full/10.1002/smtd.202500082).